In [1]:
# #Import python packages
# import numpy as np
# import matplotlib.pyplot as plt

# #Import my functions
# import sys
# sys.path.append('/Users/bono/Desktop/gm2FieldAnalysis/MyMath')
# from avg_rules import trap_avg
# sys.path.append('/Users/bono/Desktop/gm2FieldAnalysis/DataSelection')
# from treat_data import direct_removal, repeat_averager, spike_removal, phi_selection, degree_of_outlier
# from treat_data import spike_removalB
# from my_spike import my_spike_removal
# sys.path.append('/Users/bono/Desktop/gm2FieldAnalysis/Fitters/')
# from multipoles import joe_multipoles, my_multipoles

# #specify the data dir
# data_dir = '/Users/bono/Desktop/gm2FieldAnalysis/RootToText/IO/'

In [2]:
#Import python packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import seaborn as sb

#Import my functions
import sys
sys.path.append('..')
from treat_data import direct_removal, repeat_averager, spike_removal, phi_selection
from my_spike import my_spike_removal, get_derivative

#specify the data dir
data_dir = './testdata/'

In [3]:
phi_low = -1
phi_high = 500

traps_list = []
n_events_list = []
number_of_thresholds = 200
#res_arr_old = np.array([30.72, 39.84, 29.64, 37.93, 28.20, 64.66, 54.62, 34.92, 22.41, 30.67, 63.11, 58.57, 47.69, 30.85, 18.77, 29.88, 65.70])
res_arr = np.array([28.11, 36.74, 27.03, 34.85, 25.77, 60.56, 50.77, 32.10, 19.78, 27.79, 59.03, 54.15, 44.10, 27.71, 16.26, 26.71, 61.93])
print (res_arr)



[28.11 36.74 27.03 34.85 25.77 60.56 50.77 32.1  19.78 27.79 59.03 54.15
 44.1  27.71 16.26 26.71 61.93]


In [4]:
#loop over the probes and get the trapizoidal average for each probe
for probe in range(17):
    print("starting probe{}...".format(probe))
    file_path = data_dir + 'Run3956Probe' + str(probe) + '.txt'
#     print("running on probe",probe,":", "file_path=", file_path)
      #read the data into a pandas dataframe
    df = pd.read_csv(file_path,  delimiter = ',',comment='#', names=['Phi','Frequency'])
    # Convert the df to a numpy ndarray
    data = df.values
    phi = data[:,0]
    freq = data[:,1]

            
    #Treat the data
    #Sort according to phi
    arr1inds = phi.argsort()
    phi = phi[arr1inds[::1]]
    freq = freq[arr1inds[::1]]
    #Look at sub-region around the ring
    phi, freq = phi_selection(phi,freq,phi_low,phi_high)
    #Direct outlier removal
    phi,freq = direct_removal(phi,freq,20.0)
    #Average repeated points
    phi,freq = repeat_averager(phi,freq)
    
    #scan over spike thresholds
    
    traps = np.empty(0)
    n_events = np.empty(0)
    phi_removed = np.empty(0)
    freq_removed = np.empty(0)
    
    res = res_arr[probe]
    for i in range(number_of_thresholds):
        n_sigma = 20.0*(i/number_of_thresholds)
        #Remove spikes for a range of thresholds
        #first pass
        phi_removed,freq_removed = my_spike_removal(phi,freq,n_sigma,res*n_sigma)
        #second pass
        phi_removed,freq_removed = my_spike_removal(phi_removed,freq_removed,n_sigma,res*n_sigma)
        #third pass
        phi_removed,freq_removed = my_spike_removal(phi_removed,freq_removed,n_sigma,res*n_sigma)
        
        # Calculate the average for this probe and filter iteration
        trap = trap_avg(phi_removed,freq_removed)
        #store this iteration
        traps = np.append(traps, trap)
        #Calculate the number of readings for this probe and filter iteration
        n_event = phi_removed.size
        n_events = np.append(n_events, n_event)
        
    #store a list of arrays of scan averages. 
    #each item in the list is a different probe. 
    #each element in the array corresponds to a different spike threshold 
    traps_list.append(traps)
    n_events_list.append(n_events)
    


starting probe0...


NameError: name 'trap_avg' is not defined

In [ ]:
#calculate multipoles for each threshold
mpoles_list = []
threshs = np.empty(0)
n_event_array  = np.empty(0)
DP_array = np.empty(0)
NQ_array = np.empty(0)
SQ_array = np.empty(0)
NS_array = np.empty(0)
SS_array = np.empty(0)
for i in range(number_of_thresholds):
    n_sigma = 20.0*(i/number_of_thresholds)
    
    threshs = np.append(threshs, n_sigma)
    trap_array = np.empty(0)
    total_events = 0
    for probe in range(17):
        probe_value = traps_list[probe][i]
        trap_array = np.append(trap_array, probe_value)
        total_events += n_events_list[probe][i]
    mpoles = my_multipoles(trap_array,10)
    DP = mpoles.GetParameter(1)
    NQ = mpoles.GetParameter(2)
    SQ = mpoles.GetParameter(3)
    NS = mpoles.GetParameter(4)
    SS = mpoles.GetParameter(5)
    
    DP_array = np.append(DP_array, DP)
    NQ_array = np.append(NQ_array, NQ)
    SQ_array = np.append(SQ_array, SQ)
    NS_array = np.append(NS_array, NS)
    SS_array = np.append(SS_array, SS)

    n_event_array = np.append(n_event_array,total_events)
    
    



In [ ]:
# Get the multipoles without spike protection at all
#This is done poorly, but it allows independent cells, saving un-needed computing time.
#Be careful when extending this code, and changing run numbers, for example
total = 0
traps = np.empty(0)
for probe in range(17):
    file_path = data_dir + 'Run3956Probe' + str(probe) + '.txt'
    df = pd.read_csv(file_path,  delimiter = ',',comment='#', names=['Phi','Frequency'])
    # Convert the df to a numpy ndarray
    data = df.values
    phi = data[:,0]
    freq = data[:,1]
    #Sort according to phi
    arr1inds = phi.argsort()
    phi = phi[arr1inds[::1]]
    freq = freq[arr1inds[::1]]
    #Look at sub-region around the ring
    phi, freq = phi_selection(phi,freq,phi_low,phi_high)
    #Direct outlier removal
    phi,freq = direct_removal(phi,freq,20.0)
    #Average repeated points
    phi,freq = repeat_averager(phi,freq)
    
    #compute the values of interest
    total += phi.size
    trap = trap_avg(phi,freq)
    traps = np.append(traps, trap)
    #calculate the multipoles for the slice an store
mpoles = my_multipoles(traps,10)
DP = mpoles.GetParameter(1)
NQ = mpoles.GetParameter(2)
SQ = mpoles.GetParameter(3)
NS = mpoles.GetParameter(4)
SS = mpoles.GetParameter(5)

#put the desired quantities into vectors for later plotting
size = threshs.size
total_fake_vec = np.empty(size)
DP_fake_vec = np.empty(size)
NQ_fake_vec = np.empty(size)
SQ_fake_vec = np.empty(size)
NS_fake_vec = np.empty(size)
SS_fake_vec = np.empty(size)
for i in range(size):
    total_fake_vec[i] = total
    DP_fake_vec[i] = DP
    NQ_fake_vec[i] = NQ
    SQ_fake_vec[i] = SQ
    NS_fake_vec[i] = NS
    SS_fake_vec[i] = SS


In [ ]:
# font = {'family' : 'normal',
#         'weight' : 'normal',
#         'size'   : 16}
# plt.rc('font', **font)

In [ ]:
normalization = total_fake_vec[0]
print(normalization)

In [ ]:
# import mpld3 
# mpld3.disable_notebook()

# plot the result vs spike threshold
plt.plot(threshs,n_event_array/normalization, 'o')
plt.plot(threshs,total_fake_vec/normalization, '-')
title_str = "Fraction of events vs spike threshold"
# plt.title(title_str)
plt.xlabel('Threshold (N*sigma)')
plt.ylabel('Fraction of events')
plt.grid()
plt.tight_layout()
plt.savefig('N.png', dpi=2000)
plt.show()


plt.plot(threshs,DP_array, 'o')
plt.plot(threshs,DP_fake_vec, '-')
# title_str = "DP : spike threshold"
# plt.title(title_str)
plt.xlabel('Threshold (N*sigma)')
plt.ylabel('DP (Hz)')
plt.grid()
plt.tight_layout()
plt.savefig('DP.png', dpi=2000)
plt.show()

plt.plot(threshs,NQ_array, 'o')
plt.plot(threshs,NQ_fake_vec, '-')
# title_str = "NQ vs spike threshold"
# plt.title(title_str)
plt.xlabel('Threshold (N*sigma)')
plt.ylabel('NQ (Hz)')
plt.grid()
plt.tight_layout()
plt.savefig('NQ.png', dpi=2000)
plt.show()

plt.plot(threshs,SQ_array, 'o')
plt.plot(threshs,SQ_fake_vec, '-')
# title_str = "SQ vs spike threshold"
# plt.title(title_str)
plt.xlabel('Threshold (N*sigma)')
plt.ylabel('SQ (Hz)')
plt.grid()
plt.tight_layout()
plt.savefig('SQ.png', dpi=2000)
plt.show()

plt.plot(threshs,NS_array, 'o')
plt.plot(threshs,NS_fake_vec, '-')
# title_str = "NS vs spike threshold"
# plt.title(title_str)
plt.xlabel('Threshold (N*sigma)')
plt.ylabel('NS (Hz)')
plt.grid()
plt.tight_layout()
plt.savefig('NS.png', dpi=2000)
plt.show()

plt.plot(threshs,SS_array, 'o')
plt.plot(threshs,SS_fake_vec, '-')
# title_str = "SS vs spike threshold"
# plt.title(title_str)
plt.xlabel('Threshold (N*sigma)')
plt.ylabel('SS (Hz)')
plt.grid()
plt.tight_layout()
plt.savefig('SS.png', dpi=2000)
plt.show()
